In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import Figure
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.merge import merge
from pathlib import Path
import richdem as rd
from mpl_toolkits import mplot3d
from geopy import distance

In [2]:
# define constants
pi = np.pi
color_scheme = 'jet'
radius = 6371e3

In [3]:
# define dict with volc-center lat,lon,elev, and lower-left dem data corner lat,lon
volc_lat_lon = {}
volc_lat_lon['Mt_Rainier'] = [46.8528857, -121.7603744, 4392.5, 46.625, -122.000,[]]
volc_lat_lon['Mt_Adams'] = [46.202621, -121.4906384, 3743.2, 46.125, -121.625,[]]
volc_lat_lon['Mt_Baker'] = [48.7773426,  -121.8132008, 3287.6, 48.750, -121.875,[]]
volc_lat_lon['Mt_St_Helens'] =[46.200472222222224,-122.18883611111112,2549, 46.125, -122.250,[]] 
volc_lat_lon['Glacier_Peak'] = [48.1112273, -121.1139922, 3213, 48, -121.25,[]]
volc_lat_lon['Crater_Lake']=[42.907745, -122.143494, 1883, 42, -123,[]]
volc_lat_lon['Mt_Hood']=[45.373221, -121.696509, 3428.7, 45, -122,[]]
volc_lat_lon['Newberry']=[43.7220653, -121.2344654, 2435, 43, -122,[]]

In [4]:
volc_grid = {'Mt_Rainier': [46.74032510548371, -121.92393623921424, 24999.999843],
 'Mt_Adams': [46.106704928338665, -121.62841697638393, 21304.49616672],
 'Mt_Baker': [48.69314176638344, -121.94028162361221, 18711.439836960002],
 'Mt_St_Helens': [46.13517461639252, -122.28270726876059, 14507.683460400001],
 'Glacier_Peak': [48.02893063274638, -121.23658516139658, 18286.8524748],
 'Crater_Lake': [42.859490162690115, -122.20906103189014, 10717.131406800001],
 'Mt_Hood': [45.28535932500786, -121.82087738520083, 19514.51325252],
 'Newberry': [43.659665418808615, -121.32037100893913, 13858.850226]}

In [5]:
# define function to get the distance of the lower-left corner to the center of the volcano
def dist(slat,slon,elat,elon,radius):
    lam = slon-elon
    g = (np.sin(slat)*np.sin(elat))+(np.cos(slat)*np.cos(elat)*np.cos(lam))
    d = radius*np.arccos(g)
    dist = d/10
    return dist

def start_latlon(slat, slon, x, y):
    x = x*10
    y = y*10
    hypotenuse = np.sqrt((x)**2+(y)**2)
    bearing = np.arctan(x/y)
    d = distance.geodesic(meters = hypotenuse)
    slat2 = d.destination(point=[slat,slon], bearing=bearing)[0]
    slon2 = d.destination(point=[slat,slon], bearing=bearing)[1]
    return slat2, slon2

In [8]:
# merges different data files together for comprehensive data 
# only necessary for Washington Volcanoes
# for name in volc_lat_lon:
name = 'Mt_Rainier'
if volc_lat_lon[name][0] > 46:
    path = Path('Data/DEM_data/'+name)
    path1 = Path('Data/DEM_data/'+name+'/.ipynb_checkpoints')
    #path.mkdir(parents=True, exist_ok=True)
    out_path = 'Data/DEM_data/'+str(name)+'/'+str(name)+'1.tif'
    volc_files = list(path.iterdir())
    to_mosiac = []
    for d in volc_files:
        if d != path1:
            raster = rio.open(d)
            to_mosiac.append(raster)
    mosaic, output = merge(to_mosiac)
    output_meta = raster.meta.copy()
    output_meta.update(
        {'driver': 'GTiff',
            'height': mosaic.shape[1],
            'width': mosaic.shape[2],
            'transform': output,})
    with rio.open(out_path, 'w', **output_meta) as m:
        m.write(mosaic)